## **Dependencies**

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

## **Load JSON**

Choose respective json file to transform to data frame
Json be found in /src/response_[ORG]

### Define Params

In [2]:
##########
# PARAMS #
##########

abbreviation = "wb" # abbreviation of development bank
response_folder = "../../../src/responses/"
transformed_folder = "../../../src/transformed/"
response_file = response_folder + f"response_{abbreviation}.json" # json format -> fetched in iati.ipynb
output_file = transformed_folder + f"transformed_{abbreviation}.csv"

In [3]:
#Read in data into df from json
df = pd.read_json(response_file)
df.head(1)

,sector_code,iati_identifier,title_narrative,document_link_url,reporting_org_ref,sector_vocabulary,activity_date_type,activity_status_code,description_narrative,last_updated_datetime,activity_date_iso_date,recipient_country_code,location_name_narrative,reporting_org_narrative,contact_info_organisation_narrative,recipient_region_code
0,"[21023, 41010, 74020, TI, BZ, TZ, WZ, BC, 0003...",44000-P075523,[Second Infrastructure Asset Management Project],[http://documents.worldbank.org/curated/en/997...,44000,"[1, 1, 1, 99, 99, 99, 99, 99, 98, 98, 98, 98, ...","[1, 2, 3, 4]",3,[The project's development objectives are to: ...,2024-01-27T18:19:26Z,"[2002-11-11T00:00:00Z, 2003-12-29T00:00:00Z, 2...",[WS],"[Apia, Faleolo Intl / Apia, Manono-uta, Upolu,...",[World Bank],[The World Bank],NaN


In [4]:
print(f"Entries: {len(df)}")
df.reporting_org_ref.value_counts()

Entries: 4296


44000    4296
Name: reporting_org_ref, dtype: int64

In [5]:
# create new empty df to fill with transformed data

trans_df = pd.DataFrame()

## **Feature Transformation & Engineering**

### IATI ID

In [6]:
trans_df["iati_id"] = df["iati_identifier"].values

trans_df.head(1)

,iati_id
0,44000-P075523


### Title

#### Title EN

In [7]:
############
# EN title #
############

pbar = tqdm(total=len(df))
trans_df["title_en"] = "NaN"

for index, row in df.iterrows():
    title_row = row['title_narrative']
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # nan in pandas is type float
            # check if nan and if yes take first entry in lang
            if isinstance(lang_list, float):
                if isinstance(title_row, float):
                    trans_df["title_en"][index] = "NaN"
                else:
                    trans_df["title_en"][index] = title_row[0]
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if "en" or "EN" in lang_list:
                        if lang_list[j].lower() == "en":
                            title = title_row[j]
                            trans_df["title_en"][index] = title
                    else:
                        trans_df["title_en"][index] = "NaN"
            else:
                pass
        else:
            trans_df["title_en"][index] = row['title_narrative'][0]
    except:
        print(f"Error: Index: {index}, Row: {lang_list}, {title_row}")
    
    pbar.update(1)

pbar.close()

trans_df.head(1)

  0%|          | 0/4296 [00:00<?, ?it/s]

,iati_id,title_en
0,44000-P075523,Second Infrastructure Asset Management Project


#### Title Other

In [8]:
###################
# Add other title #
###################

pbar = tqdm(total=len(df))

trans_df["title_other"] = "NaN"

for index, row in df.iterrows():
    title_row = row['title_narrative']
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # every title which is has no lang attribute is classified as English and therefore not in other
            if isinstance(lang_list, float):
                trans_df["title_other"][index] = "NaN"
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if lang_list[j].lower() != "en":
                        title = row['title_narrative'][j]
                        if trans_df["title_other"][index] == "NaN":
                            trans_df["title_other"][index] = title
                        else:
                            trans_df["title_other"][index] = f"{trans_df['title_other'][index]}; {title}"
            else:
                trans_df["title_other"][index] = title
            
            pbar.update(1)

        else:
            trans_df["title_other"][index] = "NaN"
        
    except Exception as e:
        print(f"Error: Index: {index} \n Row: {row} \n Exception: {e}")

pbar.close()

trans_df.head(5)

  0%|          | 0/4296 [00:00<?, ?it/s]

,iati_id,title_en,title_other
0,44000-P075523,Second Infrastructure Asset Management Project,NaN
1,44000-P049290,Nam Theun 2 Social and Environment Project,NaN
2,44000-P074027,Health Services Improvement Project,NaN
3,44000-P086313,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,NaN
4,44000-P078113,Second Education Development Project,NaN


#### Main Title

In [9]:
trans_df['title_main'] = trans_df["title_en"]
trans_df.loc[trans_df['title_main'] == "NaN", 'title_main'] = trans_df['title_other']

trans_df.head(5)

,iati_id,title_en,title_other,title_main
0,44000-P075523,Second Infrastructure Asset Management Project,NaN,Second Infrastructure Asset Management Project
1,44000-P049290,Nam Theun 2 Social and Environment Project,NaN,Nam Theun 2 Social and Environment Project
2,44000-P074027,Health Services Improvement Project,NaN,Health Services Improvement Project
3,44000-P086313,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,NaN,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...
4,44000-P078113,Second Education Development Project,NaN,Second Education Development Project


### Organization

In [10]:
trans_df['organization'] = df['reporting_org_narrative'].apply(lambda x: x[0])

print(trans_df.organization.value_counts())
trans_df.head(2)

World Bank    4296
Name: organization, dtype: int64


,iati_id,title_en,title_other,title_main,organization
0,44000-P075523,Second Infrastructure Asset Management Project,NaN,Second Infrastructure Asset Management Project,World Bank
1,44000-P049290,Nam Theun 2 Social and Environment Project,NaN,Nam Theun 2 Social and Environment Project,World Bank


### Country

In [27]:
country_codes_df = pd.read_csv("../../../src/codelists/country_codes_ISO3166-1alpha-2.csv")
country_codes_df.head(2)

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Åland Islands,"""AX""","""ALA""","""248""","""60.116667""","""19.9"""


#### Read in country code csv

In [30]:
trans_df["country_code"] = df["recipient_country_code"]
trans_df["country"] = "NaN"

for index, row in df.iterrows():
    country_list = row["recipient_country_code"]

    if isinstance(country_list, float):
        trans_df["country"][index] = "NaN"
    else:
        country_str = ""
        for i in country_list:
            country_str += f"{i}; "
        
        trans_df["country"][index] = country_str

trans_df.head(5)

,iati_id,title_en,title_other,title_main,organization,country,region,location,description_en,description_other,country_code
0,44000-P075523,Second Infrastructure Asset Management Project,NaN,Second Infrastructure Asset Management Project,World Bank,WS;,NaN,NaN,The project's development objectives are to: (...,NaN,[WS]
1,44000-P049290,Nam Theun 2 Social and Environment Project,NaN,Nam Theun 2 Social and Environment Project,World Bank,LA;,NaN,NaN,"To generate revenues, through environmentally ...",NaN,[LA]
2,44000-P074027,Health Services Improvement Project,NaN,Health Services Improvement Project,World Bank,LA;,NaN,NaN,To assist Lao PDR to increase utilization and ...,NaN,[LA]
3,44000-P086313,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,NaN,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,World Bank,WS;,NaN,NaN,The objective of the Project is to improve the...,NaN,[WS]
4,44000-P078113,Second Education Development Project,NaN,Second Education Development Project,World Bank,LA;,NaN,NaN,Primary Enrollment and Completion increased in...,NaN,[LA]


### Region

In [12]:
trans_df['region'] = df['recipient_region_code']
trans_df.head(5)

,iati_id,title_en,title_other,title_main,organization,country,region
0,44000-P075523,Second Infrastructure Asset Management Project,NaN,Second Infrastructure Asset Management Project,World Bank,[WS],NaN
1,44000-P049290,Nam Theun 2 Social and Environment Project,NaN,Nam Theun 2 Social and Environment Project,World Bank,[LA],NaN
2,44000-P074027,Health Services Improvement Project,NaN,Health Services Improvement Project,World Bank,[LA],NaN
3,44000-P086313,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,NaN,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,World Bank,[WS],NaN
4,44000-P078113,Second Education Development Project,NaN,Second Education Development Project,World Bank,[LA],NaN


### Location

In [13]:
try: 
    if 'title_narrative_xml_lang' in df.columns:
        trans_df['location'] = df['location_name_narrative']
    else:
        trans_df['location'] = "NaN"
except:
        trans_df['location'] = "NaN"    
trans_df.head(5)

,iati_id,title_en,title_other,title_main,organization,country,region,location
0,44000-P075523,Second Infrastructure Asset Management Project,NaN,Second Infrastructure Asset Management Project,World Bank,[WS],NaN,NaN
1,44000-P049290,Nam Theun 2 Social and Environment Project,NaN,Nam Theun 2 Social and Environment Project,World Bank,[LA],NaN,NaN
2,44000-P074027,Health Services Improvement Project,NaN,Health Services Improvement Project,World Bank,[LA],NaN,NaN
3,44000-P086313,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,NaN,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,World Bank,[WS],NaN,NaN
4,44000-P078113,Second Education Development Project,NaN,Second Education Development Project,World Bank,[LA],NaN,NaN


### Descriptions

descr1, descri2, ... can have overlap. Doppelte Beschriebungen entfernen! Es gibt lib um overlap heruaszufinden!

In [18]:
#################################
# Description english and other #
#################################

trans_df["description_en"] = "NaN"
trans_df["description_other"] = "NaN"

for index, row in df.iterrows():

    try:
        if 'description_narrative_xml_lang' in df.columns:
            descr_list = row['description_narrative_xml_lang'] # list with languages provided 
            descr_row = row['description_narrative'] # list with the despription narrative of all languages provided

            # nan in pandas is type float
            # check if nan and if yes take first entry in descr
            if isinstance(descr_list, float):
                if isinstance(descr_row, float):
                    trans_df["description_en"][index] = "NaN"
                else:
                    trans_df["description_en"][index] = descr_row[0]
            else:
                if len(descr_list) == len(descr_row):
                    descr_len = len(descr_list)
                else:
                    descr_len = len(descr_row)
                # iterate throug description list
                for j in range(0, descr_len):
                    # if description english
                    if descr_list[j].lower() == "en":
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_en"][index] == "NaN":
                            trans_df["description_en"][index] = descr
                        else:
                            trans_df["description_en"][index] = f"{trans_df['description_en'][index]}; {descr}"
                    else:
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_other"][index] == "NaN":
                            trans_df["description_other"][index] = descr
                        else:
                            trans_df["description_other"][index] = f"{trans_df['description_other'][index]}; {descr}"
        else:
            descr_str = ""
            for d in row['description_narrative']:
                descr_str += f"{d}; "
            trans_df["description_en"][index] = descr_str

    except Exception as e:
        print(f"⚠︎ Error {e}: Index: {index}")
        print(row)
        
trans_df.head(5)

⚠︎ Error 'float' object is not iterable: Index: 169
sector_code                            [ET, 000065, 000656, 000658, 000811, 000081, 0...
iati_identifier                                                            44000-P179154
title_narrative                        [Tertiary Education, Science, and Technology P...
document_link_url                      [http://documents.worldbank.org/curated/en/099...
reporting_org_ref                                                                  44000
sector_vocabulary                                   [99, 98, 98, 98, 98, 98, 98, 98, 98]
activity_date_type                                                          [1, 2, 3, 4]
activity_status_code                                                                   2
description_narrative                                                                NaN
last_updated_datetime                                               2024-01-28T01:28:15Z
activity_date_iso_date                 [2023-12-08T00:00:0

,iati_id,title_en,title_other,title_main,organization,country,region,location,description_en,description_other
0,44000-P075523,Second Infrastructure Asset Management Project,NaN,Second Infrastructure Asset Management Project,World Bank,[WS],NaN,NaN,The project's development objectives are to: (...,NaN
1,44000-P049290,Nam Theun 2 Social and Environment Project,NaN,Nam Theun 2 Social and Environment Project,World Bank,[LA],NaN,NaN,"To generate revenues, through environmentally ...",NaN
2,44000-P074027,Health Services Improvement Project,NaN,Health Services Improvement Project,World Bank,[LA],NaN,NaN,To assist Lao PDR to increase utilization and ...,NaN
3,44000-P086313,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,NaN,SAMOA HEALTH SECTOR MANAGEMENT PROGRAM SUPPORT...,World Bank,[WS],NaN,NaN,The objective of the Project is to improve the...,NaN
4,44000-P078113,Second Education Development Project,NaN,Second Education Development Project,World Bank,[LA],NaN,NaN,Primary Enrollment and Completion increased in...,NaN


#### Main desription

In [ ]:
trans_df['description_main'] = trans_df.description_en
trans_df.loc[trans_df['description_main'] == "NaN", 'description_main'] = trans_df['description_other']

trans_df.head(5)

### Status

In [ ]:
# https://iatistandard.org/en/iati-standard/203/codelists/activitystatus/
activity_status = {
    1: "Pipeline/identification",
    2: "Implementation",
    3: "Finalisation",
    4: "Closed",
    5: "Cancelled",
    6: "Suspended"
}

trans_df["status"] = df.activity_status_code
trans_df['status'] = trans_df['status'].replace(activity_status)

trans_df.head(2)
    

### Date

In [ ]:
#############
# Date Type #
#############

# One Hot
# 1 -> Yes
# 0 -> No

# Codes:
# 1 Planned start
# 2 Actual start
# 3 Planned end
# 4 Actual end

trans_df["planned_start"] = "NaN"
trans_df["actual_start"] = "NaN"
trans_df["planned_end"] = "NaN"
trans_df["actual_end"] = "NaN"

date_types = {
    1: "planned_start",
    2: "actual_start",
    3: "planned_end",
    4: "actual_end"
}

for index, row in df.iterrows():
    dtype_list = row["activity_date_type"]
    iso_date_list = row["activity_date_iso_date"]

    combined_list = list(zip(dtype_list, iso_date_list))

    # replace nums with column names from date_types
    combined_list = [(date_types[int(t[0])], t[1]) for t in combined_list]

    for i in combined_list:
        trans_df[i[0]] = i[1]

trans_df.head(1800)


### Last Update

In [ ]:
trans_df['last_update'] = df['last_updated_datetime']
trans_df.head(2)

### Sector Codes (CRS & SGDs)

In [ ]:
sector_codes = {
    1: "OECD DAC CRS Purpose Codes (5 digit)",
    2: "OECD DAC CRS Purpose Codes (3 digit)",
    3: "Classification of the Functions of Government (UN)",
    4: "Statistical classification of economic activities in the European Community",
    5: "National Taxonomy for Exempt Entities (USA)",
    6: "AidData",
    7: "SDG Goal",
    8: "SDG Target",
    9: "SDG Indicator",
    10: "Humanitarian Global Clusters (Inter-Agency Standing Committee)",
    11: "North American Industry Classification System (NAICS)",
    12: "UN System Function",
    99: "Reporting Organisation", # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity
    98: "Reporting Organisation 2" # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity (if they are referencing more than one)
}

In [ ]:
crs5_df = pd.read_csv("../../../src/codelists/crs5_codes.csv")
crs5_df.head(2)

In [ ]:
crs3_df = pd.read_csv("../../../src/codelists/crs3_codes.csv")
crs3_df.head(2)

#### Get CRS5 & derive crs3 tags from crs5 tags

- other tags are not given by organizations in IATI data; need other ways to find out these tags


- It is possible to derive CRS3 tags from CRS5 tags by looking at the first 3 characters.

In [ ]:
# to process different variants of codes (crs3, crs5 etc.)

def process_codes(combined_list, translation_df, code_index):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"
    
def derive_crs3(combined_list, translation_df, code_index="1"):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1][:3]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1][:3]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"

In [ ]:
###############
# ECTRACT CRS #
###############
trans_df["crs_5_code"] = "NaN"
trans_df["crs_5_name"] = "NaN"

# Most Project dont have information on crs3 -> crs3 derived from crs5 tags
trans_df["crs_3_code"] = "NaN"
trans_df["crs_3_name"] = "NaN"


for index, row in df.iterrows():
    crs_voc_list = row['sector_vocabulary']
    crs_code_list = row['sector_code']

    if type(crs_voc_list) == float:
        pass
    else:
        try:
            combined_list = list(zip(crs_voc_list, crs_code_list))

            # CRS 5
            crs5_str, crs5_codes = process_codes(combined_list, crs5_df, "1")
            trans_df["crs_5_code"][index] = crs5_codes
            trans_df["crs_5_name"][index] = crs5_str
            
            # CRS 3
            crs3_str, crs3_codes = derive_crs3(combined_list, crs3_df)
            trans_df["crs_3_code"][index] = crs3_codes
            trans_df["crs_3_name"][index] = crs3_str

        except:
            print(f"Error on Index {index}, {crs_code_list}")
            pass

trans_df.head(5)

### Documents

In [ ]:
#df.document_link_url.value_counts()

try:
    trans_df['docs'] = df['document_link_url']
except:
    trans_df["docs"] = "NaN"
trans_df.head(2)

## **Save final DF as CSV**

In [ ]:
trans_df.head()

In [ ]:
trans_df.to_csv(output_file, index=False) 